In [1]:
import keras
from keras import layers
from keras import initializers
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from skimage import measure
from scipy.spatial.distance import cdist
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import cv2

In [2]:
# Load the model from the .keras file
model = load_model(r"C:\Users\Mafe Valenzuela\Documents\Cps\Códigos\model.keras")

In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define image dimensions
IMG_HEIGHT = 176
IMG_WIDTH = 320

def load_images_and_masks_from_directory(validation_dir):
    images = []
    masks = []
    
    # Define folder paths
    image_dir = os.path.join(validation_dir, 'images')
    mask_dir = os.path.join(validation_dir, 'masks')

    # Check if the folders exist
    if not os.path.exists(image_dir):
        print(f"Error: Image folder not found at {image_dir}")
        return np.array([]), np.array([])
    
    if not os.path.exists(mask_dir):
        print(f"Error: Mask folder not found at {mask_dir}")
        return np.array([]), np.array([])
        
    # Get sorted image and mask filenames
    image_files = sorted(os.listdir(image_dir), key=lambda x: int(os.path.splitext(x)[0]))
    mask_files = sorted(os.listdir(mask_dir), key=lambda x: int(os.path.splitext(x)[0]))

    # Check if files exist
    if not image_files:
        print("Error: No images found in the 'images' folder")
        return np.array([]), np.array([])
    
    if not mask_files:
        print("Error: No masks found in the 'masks' folder")
        return np.array([]), np.array([])
        
    # Load images and masks
    for img_file, mask_file in zip(image_files, mask_files):
        img_path = os.path.join(image_dir, img_file)
        mask_path = os.path.join(mask_dir, mask_file)

        img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        mask = load_img(mask_path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode="grayscale")

        # Convert to arrays and normalize
        img_array = img_to_array(img) / 255.0
        mask_array = img_to_array(mask) / 255.0

        images.append(img_array)
        masks.append(mask_array)

    return np.array(images), np.array(masks)

In [5]:
# Convert a prediction array into a binary mask using a threshold.
# Pixels greater than 'th' are set to 1, and the rest to 0.
def convert_to_binary(predictions, th):
    predictions = np.array(predictions) 
    binary_predictions = (predictions > th).astype(np.uint8)  
    return binary_predictions

In [6]:
# Compute the Dice coefficient, a metric for evaluating segmentation accuracy.
# It measures the overlap between the predicted and ground truth masks.
def dice_coefficient(y_true, y_pred):
    y_true_f = y_true.flatten()  
    y_pred_f = y_pred.flatten()  
    intersection = np.sum(y_true_f * y_pred_f)  
    dice = (2. * intersection) / (np.sum(y_true_f) + np.sum(y_pred_f))  # Dice formula
    return dice  # Return the Dice coefficient

In [7]:
# Create a data generator using TensorFlow's Dataset API.
# This function takes image and mask arrays, shuffles them, 
# groups them into batches, and enables infinite iteration.

def data_generator(images, masks, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((images, masks)) 
    dataset = dataset.shuffle(buffer_size=100).batch(batch_size).repeat()  
    return dataset  

In [8]:
# Computes the Mean Surface Distance (MSD), which measures the average difference 
# between the contours of the ground truth segmentation and the predicted segmentation.
def mean_surface_distance(y_true, y_pred, th):
    y_pred_binary = convert_to_binary(y_pred, th)
    y_true_binary = convert_to_binary(y_true, th)
    
    # Find the contours of the segmentations
    contours_true = measure.find_contours(y_true_binary, level=0.5)
    contours_pred = measure.find_contours(y_pred_binary, level=0.5)
    
    if len(contours_true) == 0 or len(contours_pred) == 0:
        return float('inf')  # Returns infinity if no contours are detected
    
    distances = []
    
    # Compute the minimum distance from each point in the ground truth contour to the predicted contour
    for contour_true in contours_true:
        distances.extend(np.min(cdist(contour_true, np.vstack(contours_pred), metric='euclidean'), axis=1))
    
    # Compute the minimum distance from each point in the predicted contour to the ground truth contour
    for contour_pred in contours_pred:
        distances.extend(np.min(cdist(contour_pred, np.vstack(contours_true), metric='euclidean'), axis=1))
    
    return np.mean(distances) if distances else float('inf')  # Returns the average of the distances

In [10]:
import numpy as np

def calculate_AEr(pred_mask, true_mask, pixel_size):
    """
    Computes the Absolute Error (AEr) between a predicted segmentation mask and the ground truth.

    Parameters:
    - pred_mask (numpy array): The predicted binary segmentation mask.
    - true_mask (numpy array): The ground truth binary segmentation mask.
    - pixel_size (float): The physical size of a pixel, used for area calculation.

    Returns:
    - AEr (float): The absolute error in terms of area.
    """

    # Convert to boolean arrays for logical operations
    pred_mask = np.array(pred_mask, dtype=bool)
    true_mask = np.array(true_mask, dtype=bool)
    
    # False positives: Pixels predicted as 1 but should be 0
    false_positives = np.sum(pred_mask & ~true_mask)
    
    # False negatives: Pixels predicted as 0 but should be 1
    false_negatives = np.sum(~pred_mask & true_mask)
    
    # Compute AEr
    AEr = pixel_size * (false_positives + false_negatives)
    
    return AEr

In [11]:
# valuates the Absolute Error (AEr) over a dataset using a model's predictions.
def evaluate_AEr(generator, num_steps, th, pixel_size):
    
    AEr_values = []
    iterator = iter(generator)
    
    for _ in range(num_steps):
        images, masks = next(iterator)
        
        # Ensure images and masks are numpy arrays
        images = np.array(images)
        masks = np.array(masks)
        
        # Perform model predictions
        predictions = model.predict(images)
        
        for i in range(images.shape[0]):
            y_true = np.squeeze(masks[i]).astype(np.float32)
            y_pred = np.squeeze(predictions[i]).astype(np.float32)
            
            # Convert prediction to binary using the given threshold
            y_pred_binary = convert_to_binary(y_pred, th)
            
            # Ignore images with completely black segmentations
            if np.sum(y_pred_binary) > 0:
                # Compute Absolute Error (AEr)
                AEr_value = calculate_AEr(y_pred_binary, y_true, pixel_size)
                AEr_values.append(AEr_value)
    
    # Compute the mean AEr
    mean_AEr = np.mean(AEr_values) if AEr_values else float('inf')
    
    return mean_AEr

In [14]:
# Computes a confidence interval using the bootstrap resampling method.
# Generates multiple bootstrap samples, computes their means, and determines the confidence interval based on percentiles.
def bootstrap_ci(data, num_samples=1000, confidence=0.95):

    data = np.array(data)
    sample_means = []

    # Generate bootstrap samples and compute the mean for each sample
    for _ in range(num_samples):
        sample = np.random.choice(data, size=len(data), replace=True)  # Resampling with replacement
        sample_means.append(np.mean(sample))

    # Compute percentiles to obtain the confidence interval
    lower_bound = np.percentile(sample_means, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(sample_means, (1 + confidence) / 2 * 100)
    mean = np.mean(sample_means)

    return mean, lower_bound, upper_bound


In [27]:
# Evaluate segmentation performance using the Dice coefficient.
# Iterates through a dataset, computes predictions, and calculates Dice scores.
def evaluate_metrics(generator, num_steps, th):
    dice_scores = [] 
    
    iterator = iter(generator)  # Convert generator to iterator
    
    for _ in range(num_steps):
        images, masks = next(iterator)  # Get a batch of images and masks
        predictions = model.predict(images)  # Generate predictions
        
        for i in range(images.shape[0]):
            y_true = masks[i].numpy().astype(np.float32) 
            y_pred = predictions[i].astype(np.float32)  

            assert y_true.shape == y_pred.shape, f"Shape mismatch: {y_true.shape} vs {y_pred.shape}"

            y_pred_binary = (y_pred > th).astype(np.uint8)  # Apply threshold to get binary mask

            if np.sum(y_pred_binary) > 0:  # Only compute Dice if segmentation exists
                dice = dice_coefficient(y_true, y_pred_binary)
                dice_scores.append(dice)
    
    mean_dice, lower_dice, upper_dice =bootstrap_ci(dice_scores)  # Compute mean Dice coefficient 
    
    return mean_dice, lower_dice, upper_dice # Return the average Dice score



In [28]:
def evaluate_additional_metrics(generator, num_steps, th):
    
    msd_values = []
    
    iterator = iter(generator)
    
    for _ in range(num_steps):
        images, masks = next(iterator)
        predictions = model.predict(images)
        
        for i in range(images.shape[0]):
            y_true = np.squeeze(masks[i]).astype(np.float32)
            y_pred = np.squeeze(predictions[i]).astype(np.float32)
            
            # Convert prediction to binary
            y_pred_binary = convert_to_binary(y_pred, th)
            
            # Ignore images without segmentation
            if np.sum(y_pred_binary) > 0:
                msd = mean_surface_distance(y_pred_binary, y_true, th)  # Ensure this function is defined
                msd_values.append(msd)
    
    return bootstrap_ci(msd_values)

# Evaluates Area Error Rate (AEr) with confidence intervals.
def evaluate_AEr(generator, num_steps, th, pixel_size):

    AEr_values = []
    
    iterator = iter(generator)
    
    for _ in range(num_steps):
        images, masks = next(iterator)
        predictions = model.predict(images)
        
        for i in range(images.shape[0]):
            y_true = np.squeeze(masks[i]).astype(np.float32)
            y_pred = np.squeeze(predictions[i]).astype(np.float32)
            
            # Convert prediction to binary
            y_pred_binary = convert_to_binary(y_pred, th)
            
            # Ignore images without segmentation
            if np.sum(y_pred_binary) > 0:
                AEr = calculate_AEr(y_pred_binary, y_true, pixel_size)
                AEr_values.append(AEr)
    
    return bootstrap_ci(AEr_values)

In [ ]:
train_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\train"
X_train, y_train = load_images_and_masks_from_directory(train_dir)

validation_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\val"
X_val, y_val = load_images_and_masks_from_directory(validation_dir)

test_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\test"
X_test, y_test = load_images_and_masks_from_directory(test_dir)

In [30]:
BATCH_SIZE = 8
pixel_size = 0.16
th=0.45

train_gen = data_generator(X_train, y_train, BATCH_SIZE)
val_gen = data_generator(X_val, y_val, BATCH_SIZE)
test_gen = data_generator(X_test, y_test, BATCH_SIZE)

In [31]:
train_steps = len(X_train)//BATCH_SIZE
val_steps = len(X_val)//BATCH_SIZE
test_steps = len(X_test)//BATCH_SIZE

*Cálculo de resultados*

In [33]:
(dice_mean, dice_lower, dice_upper) = evaluate_metrics(train_gen, train_steps, th)
(msd_mean, msd_lower, msd_upper) = evaluate_additional_metrics(train_gen, train_steps, th)
(aer_mean, aer_lower, aer_upper) = evaluate_AEr(train_gen, train_steps, th, pixel_size)

print(f"Coeficiente de Dice train (95% IC): {dice_mean:.4f} ({dice_lower:.4f}, {dice_upper:.4f})")
print(f"Mean Surface Distance train (95% IC): {msd_mean:.4f} ({msd_lower:.4f}, {msd_upper:.4f})")
print(f"Área Error Rate train (95% IC): {aer_mean:.4f} ({aer_lower:.4f}, {aer_upper:.4f}) mm²")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 

In [41]:
(dice_mean, dice_lower, dice_upper) = evaluate_metrics(val_gen, val_steps, th)
(msd_mean, msd_lower, msd_upper) = evaluate_additional_metrics(val_gen, val_steps, th)
(aer_mean, aer_lower, aer_upper) = evaluate_AEr(val_gen, val_steps, th, pixel_size)

print(f"Coeficiente de Dice validacion (95% IC): {dice_mean:.4f} ({dice_lower:.4f}, {dice_upper:.4f})")
print(f"Mean Surface Distance validacion (95% IC): {msd_mean:.4f} ({msd_lower:.4f}, {msd_upper:.4f})")
print(f"Área Error Rate validacion (95% IC): {aer_mean:.4f} ({aer_lower:.4f}, {aer_upper:.4f}) mm²")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Coeficiente de Dice validacion (95% IC): 0.8418 (0.7955, 0.8781)
Mean Surface Distance validacion (95% IC): 8.2562 (6.7660, 10.0016)
Área Error Rate validacion (95% IC): 567.7764 (478.2659, 668.5594) mm²


In [39]:
(dice_mean, dice_lower, dice_upper) = evaluate_metrics(test_gen, test_steps, th)
(msd_mean, msd_lower, msd_upper) = evaluate_additional_metrics(test_gen, test_steps, th)
(aer_mean, aer_lower, aer_upper) = evaluate_AEr(test_gen, test_steps, th, pixel_size)

print(f"Coeficiente de Dice test (95% IC): {dice_mean:.4f} ({dice_lower:.4f}, {dice_upper:.4f})")
print(f"Mean Surface Distance test (95% IC): {msd_mean:.4f} ({msd_lower:.4f}, {msd_upper:.4f})")
print(f"Área Error Rate test (95% IC): {aer_mean:.4f} ({aer_lower:.4f}, {aer_upper:.4f}) mm²")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Coeficiente de Dice test (95% IC): 0.8374 (0.7808, 0.8707)
Mean Surface Distance test (95% IC): 7.2220 (6.3881, 8.1793)
Área Error Rate test (95% IC): 520.8581 (457.7123, 591.7677) mm²
